In [1]:
import pandas as pd
import random
from math import factorial as fact
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None
from collections import Counter
import time
import sys
import glob
import re
from itertools import chain
import os

# HH Reading function

In [2]:
def set_dict():
    #All the variables from our database
    mydict={}
    
    ###Table type info####
    mydict["day"]=[]
    mydict["time"]=[]
    mydict["table_type"]=[]
    mydict["buy_in"]=[]
    mydict["hand_number"]=[]
    mydict["table_tag"]=[]
    
    ####Hero cards####
    mydict["dealthero"] = []
    
    ####Board########
    mydict["flop"] = []
    mydict["turn"] = []
    mydict["river"] = []
    mydict["BB"] = []
    mydict["ante"]=[]
    mydict["pot"]=[]
    
    ####Seats info######
    for i in range(1,10):
        mydict[f"Seat {i} hand"] = []
        mydict[f"Seat {i} player"] = []
        mydict[f"Seat {i} bankroll"]=[]
        mydict[f"Seat {i} action"]=[]
        mydict[f"Seat {i} benef"]=[]
    mydict["seat BB"]=[]
    mydict["seat SB"]=[]
    
    return mydict

def HH_read(string,elmt,room="PS"):
    if string=="flop":
        s="*** FLOP *** ["
        return elmt.split("*** FLOP *** [")[1].split("]\n")[0].split(" ")
    if string=="turn":
        s="*** TURN *** ["
        return [elmt.split("*** TURN *** [")[1].split("] [")[1].split("]")[0]]
    if string=="river":
        s="*** RIVER *** ["
        return [elmt.split("*** RIVER *** [")[1].split("] [")[1].split("]")[0]]
    if string=="hero":
        s="Dealt to bowaka ["
        return elmt.split("Dealt to bowaka [")[1].split("]\n")[0].split(" ")
    if string =="BB":
        if "Level" in elmt:
            r = elmt.split("Level")[1].split(" ")[2].strip("()").split("/")[1]
        else:
            if "Hold'em No Limit (€" in elmt:
                r= elmt.split("Hold'em No Limit (€")[1].split("/€")[1].split(")")[0]
            if "Hold'em No Limit ($" in elmt:
                r= elmt.split("Hold'em No Limit ($")[1].split("/$")[1].split(")")[0]
        return r
    if string=="ante":
        if "posts the ante " in elmt:
            r=elmt.split("posts the ante ")[1].split("\n")[0]
        else:
            r=0
        return r


def table_carac(elmt):
    #Function that complete the dictionnary for the caracteristics of the hand (cards dealt to hero, cards on flop, BB, SB...)        
        ## ADDING HERO CARDS ##
        if "Dealt to bowaka [" in elmt:
            mydict["dealthero"].append(HH_read("hero",elmt))
        else:
            mydict["dealthero"].append("NaN")
        ## ADDING FLOP COLUMN ## 
        if "*** FLOP *** [" in elmt:
            mydict["flop"].append(HH_read("flop",elmt))
        else:
            mydict["flop"].append("Nan")
        ## ADDING RIVER COLUMN ##   
        if "*** TURN *** [" in elmt:
            mydict["turn"].append(HH_read("turn",elmt))
        else:
            mydict["turn"].append("NaN")
        ## ADDING RIVER COLUMN ##    
        if "*** RIVER *** [" in elmt:
            mydict["river"].append(HH_read("river",elmt))
        else:
            mydict["river"].append("NaN")
        ## ADDING BB ##    
        mydict["BB"].append(HH_read("BB",elmt))
        ## ADDING ANTE ##
        mydict["ante"].append(HH_read("ante",elmt))
            
def table_type(name,elmt):
    name=re.sub(".txt","",name)
    mydict["table_type"].append("NaN")
    mydict["day"].append("NaN")
    mydict["buy_in"].append("NaN")
    mydict["table_tag"].append("NaN")
    mydict["hand_number"].append("NaN")
    mydict["time"].append("NaN")
    if 'CET' in elmt:
        mydict["day"][-1]=elmt.split(" CET")[0].split(" ")[-2]
        mydict["time"][-1]=elmt.split(" CET")[0].split(" ")[-1] 
    else:
        mydict["day"][-1]=elmt.split(" ET")[0].split(" ")[-2]
        mydict["time"][-1]=elmt.split(" ET")[0].split(" ")[-1]       
    if "EUR" not in name and "USD" not in name:
        mydict["table_type"][-1]="Tournament"
        if "Zoom" in name:
            mydict["table_type"][-1]="Zoom Tournament"
    else:
        if "Zoom" not in elmt:
            mydict["table_type"][-1]="Cash Game"
        else:
            mydict["table_type"][-1]="Zoom Cash Game"
    buyin=[word for word in name.split(" ") if ("€" in word or "$" in word)]
    if mydict["table_type"][-1]=="Cash Game" or mydict["table_type"][-1]=="Zoom Cash Game":
        if "€" in buyin[0]:
            mydict["buy_in"][-1]="NL"+str(int(float(buyin[0].split("-€")[1])*100))
        if "$" in buyin[0]:
            mydict["buy_in"][-1]="NL"+str(int(float(buyin[0].split("-$")[1])*100))
    else:
        mydict["buy_in"][-1]=sum([convint(s) for s in buyin])
        
def playerhand(elmt):
    ## ADDING NAME OF EACH PLAYER AT EACH SEAT  / ADDING HANDS WHEN AVAILABLE## 
    u=elmt.split("*** SUMMARY ***\n")[1]
    for i in range(1,10):
        if f"Seat {i}" in u: #checking if the corresponding seat is in the hand
            for line in u.split("\n"):
                if f"Seat {i}" in line:
                    mydict[f"Seat {i} player"].append(line.split(" ")[2])
                    if "showed" in line.split(" "):
                        mydict[f"Seat {i} hand"].append(line.split("[")[1].split("]")[0].split(" "))
                    else:
                        mydict[f"Seat {i} hand"].append("NaN")               
        else:
            mydict[f"Seat {i} player"].append("NaN")
            mydict[f"Seat {i} hand"].append("NaN")
            
def basicstats(elmt,playerseat,playerpay):
    global pot
    global bid
    global seatBB
    global seatSB
    #Add variables
    #pname : player name
    #seat : player seat
    #BB : value of the big blind
    #SB : value of the small blind
    #ante : value of the ante
    #pot : value of the pot at the given time, to be used to asses raises % in the pot by players
    #bid : value of the current bid
    u = elmt.split("*** HOLE CARDS ***")[0].split("\n")
    for line in u[2:]:
#####################################GLOBAL INITIATION OF ALL VARIABLE AND DICTIONNARIES#########################################
################################################################################################################################
        if ("Seat" in line and "out of hand" not in line and "chips" in line ):
            chips = convint(line.split("(")[-1].split(" ")[0]) #chips of a player
            pname = findname(line) #we identify the name corresponding to the seat
            seat = line.split(": ")[0] #we identify the seat
            playerseat[pname]=seat #we associate the name and the seat and store them in a dictionary organised by name
            playerpay[pname]=0 #initation of the money lost in the hand
            action[pname]=[]
            playerbid[pname]=0 #we set the initial bid of players to 0
            mydict[f"{seat} bankroll"][-1] = chips #we associate the bankroll of the players with its seat
##################################BB CALC######################################################################################
#################################################################################################################################
        if ("posts big blind" in line and "moved from another" not in line):
            if "and is all-in" not in line:
                BB = convint(line.split(" ")[-1])
                pot += BB
                bid = BB
            else:
                BB=convint(line.split(" ")[-4])
                pot += BB
                bid = BB
            playerBB = line.split(": ")[0]
            seatBB=playerseat[playerBB] #Seat BB identification
            playerpay[playerBB]+=BB
            playerbid[playerBB]=BB
############################################################################SB CALC#################################################
#################################################################################################################################
        if ("posts small blind" in line and "moved from another" not in line):
            if "and is all-in" not in line:
                SB = convint(line.split(" ")[-1])
                pot += SB
            else:
                SB=convint(line.split(" ")[-4])
                pot += SB
            playerSB = line.split(": ")[0]                    
            seatSB=playerseat[line.split(": ")[0]] #Seat SB identification
            playerpay[playerSB]+=SB
            playerbid[playerSB]=SB
####################################################################ANTE CALC#####################################################
################################################################################################################################
        if "posts the ante" in line:
            if "and is all-in" not in line:
                ante = convint(line.split(" ")[-1])
                pot += ante
            else:
                ante=convint(line.split(" ")[-4])
                pot += ante
            playerAnte = line.split(": ")[0]
            playerpay[playerAnte]+=ante
            
def findname(s):
    i = "in chips"
    r = s.split(" (")
    if i in r[-1]:
        r=" ("+r[-1]
    else:
        r=" ("+r[-2]+" ("+r[-1]
    return s.split(r)[0].split(": ")[1]

def convint(s):
    #remove € symbole and convert to integer
    s = re.sub("\$","",s)
    return float(re.sub("€","",s))

def cutstr(line):
    a=line
    if " collected " in line:
        a=line.split(" collected ")[1]
    if "Uncalled bet (" in line:
        a=line.split("Uncalled bet (")[1].split(")")[0]
    a = re.sub(" and is all-in","",a)
    if " from side pot" in line:
        a= line.split(" from side pot")[0].split(" ")[-1]
    a= re.sub(" from pot","",a)
    a= re.sub(" from main pot","",a)
    a= re.sub(" and has reached the 0 cap","",a)
    return a
    
def chips(line,i):
    return cutstr(line).split(" ")[-i]

def read_action(s1,s2):
    global pot
    global bid
    #read the action of one step of the hand
    u=elmt.split(s1)[1].split(s2)[0].split("\n")
    if s1=="*** HOLE CARDS ***":
        a="PREFLOP"
    if s1=="*** FLOP ***":
        a="FLOP"
    if s1=="*** TURN ***":
        a="TURN"
    if s1=="*** RIVER ***":
        a="RIVER"
    for fname,val in playerbid.items():
        if s1 != "*** HOLE CARDS ***":
            playerbid[fname]=0 #we initiate for the round the bid to 0
    
    for line in u:
        if ": folds" in line:
            fname = line.split(": ")[0]
            action[fname].append((a,"F",playerbid[fname]))

        if ": raises" in line:
            fname = line.split(": ")[0]
            chipraise = convint(chips(line,3))
            action[fname].append((a,"R",chipraise+bid,bid,pot))
            pot += chipraise+bid-playerbid[fname]
            playerpay[fname] += chipraise+bid-playerbid[fname] #we add to the amound already bid this the remaining money to call the current bid
            bid = chipraise + bid              
            playerbid[fname]=bid  #we update player bid           

        if ": bets" in line:
            fname = line.split(": ")[0]
            chipraise = convint(chips(line,1))
            action[fname].append((a,"B",chipraise,pot))
            bid = chipraise
            pot += bid
            playerbid[fname]=bid
            playerpay[fname]+=bid

        if ": calls" in line:
            fname = line.split(": ")[0]
            chipcall = convint(chips(line,1))
            action[fname].append((a,"C",chipcall,pot))
            pot += chipcall
            playerpay[fname]+=chipcall #we add to the amound already bid this the remaining money to call the current bid
            playerbid[fname]=bid  #we update player bid

        if ": checks" in line:
            fname = line.split(": ")[0]
            action[fname].append((a,"X",0))

        if "Uncalled bet (" in line:
            fname = line.split("returned to ")[1]
            uncalledbet = convint(cutstr(line))
            playerpay[fname] -= uncalledbet

        if " collected " in line:
            fname = line.split(" collected ")[0]
            playerpay[fname] -= convint(chips(line,1))
            
def globalaction(elmt):
    #routine that read the HH in order to get action of each player, pot size, bid, etc..
#####################################################################################################
#############################PREFLOP ACTION##########################################################
        if "*** FLOP ***" in elmt:
            read_action("*** HOLE CARDS ***","*** FLOP ***")
            if "*** TURN ***" in elmt:
######################################################################################################
########################################################FLOP ACTION###################################
                read_action("*** FLOP ***","*** TURN ***")
######################################################################################################
########################################################TURN ACTION###################################
                if "*** RIVER ***" in elmt:
                    read_action("*** TURN ***","*** RIVER ***")
                    read_action("*** RIVER ***","*** SUMMARY ***")  ####RIVER ACTION
                else:
                    read_action("*** TURN ***","*** SUMMARY ***")    
            else:
                read_action("*** FLOP ***","*** SUMMARY ***")
        else:
            read_action("*** HOLE CARDS ***","*** SUMMARY ***")

# Creation of the dataframe

In [3]:
#imax = 1320
#playerbid : dictionnary including the latest bid of player for each round player: bid

# def read_PS_data(*filenames):

os.chdir('/home/jacky/Desktop/poker/HHtest')
filenames = glob.glob('*.txt')

mydict=set_dict()
j=0
for name in filenames[:100]:
    HH = name.split(" ")[0]
    with open(name, encoding='utf-8') as entree:
        data=entree.read().strip().split("PokerStars Hand")[1:]
        for elmt in data:
            if "Hand cancelled" in elmt:
                continue
            for i in range(1,10):

                mydict[f"Seat {i} action"].append("NaN") #we initiate the new line of seats to "NaN"
                mydict[f"Seat {i} bankroll"].append("NaN")
                mydict[f"Seat {i} benef"].append("NaN")

            mydict['pot'].append(0)   
            playerseat={} #local dict storing seat number by player name
            playerpay={}  #local dict storing amount of money player paid on a given hand
            playerbid={}
            action = {}
            pot = 0
            bid = 0
            seatBB="NaN"
            seatSB="NaN"
            table_type(name,elmt)
            table_carac(elmt)
            basicstats(elmt,playerseat,playerpay) #fullfil playerseat, BB value, SB value, Ante value and seat of BB
            globalaction(elmt) #associate the action from beginning to the end to each player in the dict action[player]
            playerhand(elmt)

            for player,move in action.items():
                seat = playerseat[player]
                mydict[f"{seat} action"][-1]=action[player] #if a player played, we associated its seat with its action
            for player,bid in playerpay.items():
                seat = playerseat[player]
                mydict[f"{seat} benef"][-1]=-playerpay[player]
            mydict["seat BB"].append(seatBB)
            mydict["seat SB"].append(seatSB)
            for key,val in playerpay.items():
                if -val>mydict["pot"][-1]:
                    mydict["pot"][-1]=-val

for key,val in mydict.items():
    print(key+" : "+str(len(mydict[key])))


pd.DataFrame(mydict,columns=mydict.keys())        

day : 83033
time : 83033
table_type : 83033
buy_in : 83033
hand_number : 83033
table_tag : 83033
dealthero : 83033
flop : 83033
turn : 83033
river : 83033
BB : 83033
ante : 83033
pot : 83033
Seat 1 hand : 83033
Seat 1 player : 83033
Seat 1 bankroll : 83033
Seat 1 action : 83033
Seat 1 benef : 83033
Seat 2 hand : 83033
Seat 2 player : 83033
Seat 2 bankroll : 83033
Seat 2 action : 83033
Seat 2 benef : 83033
Seat 3 hand : 83033
Seat 3 player : 83033
Seat 3 bankroll : 83033
Seat 3 action : 83033
Seat 3 benef : 83033
Seat 4 hand : 83033
Seat 4 player : 83033
Seat 4 bankroll : 83033
Seat 4 action : 83033
Seat 4 benef : 83033
Seat 5 hand : 83033
Seat 5 player : 83033
Seat 5 bankroll : 83033
Seat 5 action : 83033
Seat 5 benef : 83033
Seat 6 hand : 83033
Seat 6 player : 83033
Seat 6 bankroll : 83033
Seat 6 action : 83033
Seat 6 benef : 83033
Seat 7 hand : 83033
Seat 7 player : 83033
Seat 7 bankroll : 83033
Seat 7 action : 83033
Seat 7 benef : 83033
Seat 8 hand : 83033
Seat 8 player : 83033
Seat

,day,time,table_type,buy_in,hand_number,table_tag,dealthero,flop,turn,river,...,Seat 8 bankroll,Seat 8 action,Seat 8 benef,Seat 9 hand,Seat 9 player,Seat 9 bankroll,Seat 9 action,Seat 9 benef,seat BB,seat SB
0,2018/07/21,12:17:29,Tournament,0,NaN,NaN,NaN,"[8c, 5d, 5h]",[3c],[8h],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Seat 4,Seat 3
1,2018/07/21,12:17:29,Tournament,0,NaN,NaN,NaN,"[Qc, 7d, 4d]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Seat 5,Seat 4
2,2018/07/21,12:17:29,Tournament,0,NaN,NaN,NaN,"[Ah, 2h, 3s]",[Ac],[9s],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Seat 3,Seat 2
3,2018/07/21,12:17:29,Tournament,0,NaN,NaN,NaN,Nan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Seat 2,Seat 1
4,2018/07/21,12:17:30,Tournament,0,NaN,NaN,NaN,Nan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Seat 2,Seat 1
5,2018/07/21,12:17:30,Tournament,0,NaN,NaN,NaN,"[5s, 6c, 9c]",[8s],[Jd],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Seat 6,Seat 5
6,2018/07/21,12:17:30,Tournament,0,NaN,NaN,NaN,Nan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Seat 2,Seat 1
7,2018/07/21,12:17:30,Tournament,0,NaN,NaN,NaN,"[Ks, 2c, 2s]",[8s],NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Seat 2,Seat 1
8,2018/07/21,12:17:30,Tournament,0,NaN,NaN,NaN,Nan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Seat 4,Seat 1
9,2018/07/21,12:17:30,Tournament,0,NaN,NaN,NaN,Nan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Seat 3,Seat 2


'141.txt'